In [ ]:
import sys
sys.path.append('./modules/')
from testing import testModel
from helpers import prepareRun 

import tensorflow as tf

import tflearn
from tflearn.layers import batch_normalization, local_response_normalization
from tflearn.data_utils import image_preloader
from tflearn.optimizers import momentum, Adam
from tflearn.activations import softmax, relu
from tflearn.layers.core import input_data, dropout
from tflearn.layers import max_pool_2d, regression, conv_2d, fully_connected
from tflearn.data_augmentation import * 
from tflearn.data_preprocessing import *
tf.GraphKeys.LAYER_TENSOR = 'layer_tensor'

In [ ]:
# Generate Folder Structure
struct = prepareRun('Final')
# Load the training data
x, y = image_preloader('./Data/train/training.tsv', categorical_labels=True,
                              image_shape=[32, 32, 3], mode='file', normalize=True)

In [ ]:
# Create augmentation pipeline with flipping on both axis and then random cropping
imgAug = ImageAugmentation()
imgAug.add_random_flip_leftright()
imgAug.add_random_flip_updown()
imgAug.add_random_crop((28, 28))

# Create prep pipeline - crop (already done on the training set) and add ZCA whitening
imgPrep = ImagePreprocessing()
imgPrep.add_crop_center((28, 28))
imgPrep.add_zca_whitening

# Model constructor
def buildModel():
    network = input_data(shape=[None, 28, 28, 3], data_augmentation=imgAug, data_preprocessing=imgPrep)
    # All conv blocks follow the format with slowly increasing filter depths:
    # Convolution - BN - RELU - Convolution - BN - RELU - FMP
    
    # CNN Block 1
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 2
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 3
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 4
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # FC Block 1
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='relu')
    network = batch_normalization(network)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='Adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.05)
    return network

# Initialize the model and build the trainer:
net = buildModel()
model = tflearn.DNN(net, max_checkpoints=10, tensorboard_verbose=0,
                checkpoint_path=struct['Checkpoints'], tensorboard_dir=struct['Tensorboard'])

# Train the model - keep 10% (or 5000 images) as a validation set to gauge training effectiveness
model.fit(x, y, n_epoch=256, 
              validation_set=0.10, 
              shuffle=True, 
              show_metric=True, 
              batch_size=100,
              snapshot_step=1000, 
              snapshot_epoch=False, run_id=struct['run'])

# Save the model
model.save('./Final/model.tflearn')

In [ ]:
# Single pass results - multipass moved to its own workbook for clarity
results = testModel(model, './Data/test/*.png', struct['base'], struct['run'], save_to_csv=True)

In [ ]:
# 4th layer w 10 pass eval:
# https://www.kaggle.com/c/cifar-10/leaderboard?submissionId=3638230
# 88.10